# Problem 4 - Largest Palindrome Product
A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99. Find the largest palindrome made from the product of two 3-digit numbers.
## My Working
First, we need a way to test if a given number, $n$, is a Palindrome.

In [2]:
def isPalindrome(n):
    nStr = str(n)
    nMid = int(len(str(n))/2)
    return nStr[:nMid] == nStr[::-1][:nMid]

Second, we try a brute force method: testing all possible products of numbers between $1$ and $m$

In [15]:
def largestPalindrome(m):
    start = 10**(len(str(m))-1)  #Find the smallest number with the same number of digits as m
    palindromes = [i * j for i in range(start, m + 1) for j in range(start, m + 1) if isPalindrome(i * j)]
    return max(palindromes)

In [16]:
print (largestPalindrome(99))
%timeit largestPalindrome(99)

9009
7.19 ms ± 407 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
print (largestPalindrome(999))
%timeit largestPalindrome(999)

906609
701 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This brute force function returns the correct answer, but is slowed by 2 orders of magnitude. Savings may be made by:
<ol>
    <li>Testing products in order of decreasing size and stopping at the first palindrome</li>
    <li>Avoid double testing (e.g. $1*2$ and $2*1$)</li>
    <li>Recognising that all palindromes are multiples of $11$ - if we are testing $i*j$, we can ignore all products where $i$ is not a multiple of $11$.</li>
    </ol>

We create a multiplication array to test how the product of two numbers vary in size.

In [11]:
import numpy as np
df = a = np.zeros(shape=(10,10))
for i in range(1,11):
    for j in range(1,11):
        df[i-1,j-1]=i*j

print (df)

[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
 [  2.   4.   6.   8.  10.  12.  14.  16.  18.  20.]
 [  3.   6.   9.  12.  15.  18.  21.  24.  27.  30.]
 [  4.   8.  12.  16.  20.  24.  28.  32.  36.  40.]
 [  5.  10.  15.  20.  25.  30.  35.  40.  45.  50.]
 [  6.  12.  18.  24.  30.  36.  42.  48.  54.  60.]
 [  7.  14.  21.  28.  35.  42.  49.  56.  63.  70.]
 [  8.  16.  24.  32.  40.  48.  56.  64.  72.  80.]
 [  9.  18.  27.  36.  45.  54.  63.  72.  81.  90.]
 [ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100.]]


We note that we:
<ul>
    <li>need only consider one half of the matrix to avoid double counting</li>
    <li>must follow a non-trivial (diagonal-esque) pattern to follow products in order of decreasing value</li>
    <li>could reduce the number of elements along one axis of the matrix by factor $11$</li>
    </ul>

## The model answer


In [20]:
def largestPalindromeModelAnswer():
    largest = 0
    a = 999
    while a >= 100:
        if a % 11 == 0:
            b = 999
            db = 1
        else:
            b = 990
            db = 11
            
        while b >= a:
            if a*b <= largest:
                break #Since a*b is always going to be too small

            if isPalindrome(a*b):
                largest = a*b

            b = b-db

        a = a-1

    return largest

print (largestPalindromeModelAnswer())

906609


In [21]:
%timeit largestPalindromeModelAnswer()

635 µs ± 53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The model answer addresses two of the possible improvements, namely the double-counting and the multiples of $11$. We might attempt to beat the model answer by cracking the non-trivial navigation through pairs...

## Beating the model answer